In [ ]:
import os
import pandas as pd
import time
import re
import itertools
import plotly

from datetime import datetime as dt
from API_Intergration import expected_modeling_tags

plotly.io.renderers.default = 'browser'

pd.options.plotting.backend = 'plotly'


''' Enter the path of your file '''
coeffs = pd.read_excel("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Co-Effs\\Coronal Co-eff.xlsx")

In [ ]:
plant = 'Latitude'
start = dt(2022,8,1)
end = dt(2022,9,1)

In [ ]:
plant_coeffs = coeffs[coeffs["Plants"] == plant]
plant_coeffs

In [ ]:
poa = plant_coeffs["POA"].values
poa2 = plant_coeffs["POA^2"].values
poa_tamb = plant_coeffs["POA*TAMB"].values
poa_ws = plant_coeffs["POA*WS"].values
derate_factor = plant_coeffs["Factor"].values
ac_loss = plant_coeffs["AC loss"].values
dc_capacity = plant_coeffs["DC_CAP(kW)"].values
clipping_setpoint = plant_coeffs["Clipping Set Points kW"].values

In [ ]:
df = expected_modeling_tags(plant,start,end)
df.filter().isin()

In [ ]:
df = df.set_index([df.index])
df.index = pd.to_datetime(df.index)
df = df[~df.index.duplicated(keep='first')]
df = df[:-1]

In [ ]:

poa_sensor= df.filter(regex = 'IRRADIANCE_POA')
poa_names = list(poa_sensor)

df[:,"POA"] = df.loc[:, poa_sensor.isin(poa_names)]

In [ ]:
df.loc[:,'POA'] = df.filter(regex = 'IRRADIANCE_POA').median(axis = 1)
df.loc[:,'WS'] = df.filter(regex = 'WIND_SPEED').median(axis = 1)
df.loc[:,'T_AMB'] = df.filter(regex = 'T_AMB').median(axis = 1)
df.loc[:,'Meter_Power'] = (df.filter(regex = 'AC_POWER').median(axis = 1)).clip(0)
df

In [ ]:
df.loc[:,'Expected_Power_V1'] = (derate_factor*ac_loss*df["POA"]*(poa + (poa2*df['POA']) + (df['T_AMB']*poa_tamb) + (df['WS']*poa_ws))).clip(0,int(clipping_setpoint))

In [ ]:
df1 = df[['Meter_Power','Expected_Power_V1']]
fig = df1.plot.line(title = 'Actual vs Expected Power kW')
fig.show()

In [ ]:
DE = df1.resample('1D').sum()/12000
DE.loc[:,'Loss_V1'] = DE['Expected_Power_V1'] - DE['Meter_Power']
DE
print(DE.Expected_Power_V1.sum())
DE.Meter_Power.sum()